In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
pd.set_option('display.max_columns', None)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [27]:
# load DRG and ICD10 lookup files and carier data file
## refer to "inpatient_data_preprocessing_.ipynb"
LU_drg = pd.read_csv('/content/drive/MyDrive/Data/Input/DRG.csv')[['DRG', 'Description']]
LU_ICD = pd.read_csv('/content/drive/MyDrive/Data/icd10/ICD10Diagnosis.csv')

# organize (renaming; file types, etc.)
LU_drg.rename(columns={'Description': 'DRG_Description'}, inplace=True)
LU_drg['DRG'] = LU_drg['DRG'].fillna(0).astype(int).astype(str).str.zfill(3)
LU_ICD.rename(columns={'Code': 'ICD_DIAG_CD', 'Description': 'ICD_Description'}, inplace=True)

professional = pd.read_csv('/content/drive/MyDrive/Data/Input/carrier.csv', sep="|")
col_professional = pd.DataFrame(professional.dtypes, columns=['type'])
#col_professional

<ipython-input-27-fee0491579b6>:11: DtypeWarning: Columns (13,22,52,53,94) have mixed types. Specify dtype option on import or set low_memory=False.
  professional = pd.read_csv('/content/drive/MyDrive/Data/Input/carrier.csv', sep="|")


In [28]:
professional['LINE_PLACE_OF_SRVC_CD'].value_counts()

,count
LINE_PLACE_OF_SRVC_CD,
11,926479
20,173831
22,11448
12,3850
31,3400
34,1955
2,41


* Count only encounters from office visits (11), so as not to double-count inpatients, outpatients, etc. [See pg. 218 of the [fee-for-service codebook](https://www2.ccwdata.org/documents/10280/19022436/codebook-ffs-claims.pdf)]
* This assumes that in/out-patient encounters have already been counted in other analysis files - "inpatient_data_preprocessing_.ipynb" and "outpatient_data_preprocessing_.ipynb"

In [29]:
professional = professional[professional['LINE_PLACE_OF_SRVC_CD'] == 11]

In [30]:
# convert columns to appropriate data types
professional['BENE_ID'] = professional['BENE_ID'].astype(str)
professional['CLM_ID'] = professional['CLM_ID'].astype(str)
professional['CLM_FROM_DT'] = pd.to_datetime(professional['CLM_FROM_DT'], format='%d-%b-%Y')
professional['CLM_THRU_DT'] = pd.to_datetime(professional['CLM_THRU_DT'], format='%d-%b-%Y')

# compute claim_days (similar to LOS) and YR
professional['CLAIM_DAYS'] = (professional['CLM_THRU_DT'] - professional['CLM_FROM_DT']).dt.days + 1
professional['YR'] = professional['CLM_FROM_DT'].dt.year

professional['CLAIM_DAYS'].value_counts()

,count
CLAIM_DAYS,
1,901926
2,24553


In [31]:
# filter for only CLAIM_DAYS=1 (ignoring billings of 2 or more days)
professional = professional[professional['CLAIM_DAYS'] == 1]
professional = professional.drop(columns=['CLAIM_DAYS'])
professional

,BENE_ID,CLM_ID,NCH_NEAR_LINE_REC_IDENT_CD,NCH_CLM_TYPE_CD,CLM_FROM_DT,CLM_THRU_DT,NCH_WKLY_PROC_DT,CARR_CLM_ENTRY_CD,CLM_DISP_CD,CARR_NUM,CARR_CLM_PMT_DNL_CD,CLM_PMT_AMT,CARR_CLM_PRMRY_PYR_PD_AMT,RFR_PHYSN_UPIN,RFR_PHYSN_NPI,CARR_CLM_PRVDR_ASGNMT_IND_SW,NCH_CLM_PRVDR_PMT_AMT,NCH_CLM_BENE_PMT_AMT,NCH_CARR_CLM_SBMTD_CHRG_AMT,NCH_CARR_CLM_ALOWD_AMT,CARR_CLM_CASH_DDCTBL_APLD_AMT,CARR_CLM_HCPCS_YR_CD,CARR_CLM_RFRNG_PIN_NUM,PRNCPAL_DGNS_CD,PRNCPAL_DGNS_VRSN_CD,ICD_DGNS_CD1,ICD_DGNS_VRSN_CD1,ICD_DGNS_CD2,ICD_DGNS_VRSN_CD2,ICD_DGNS_CD3,ICD_DGNS_VRSN_CD3,ICD_DGNS_CD4,ICD_DGNS_VRSN_CD4,ICD_DGNS_CD5,ICD_DGNS_VRSN_CD5,ICD_DGNS_CD6,ICD_DGNS_VRSN_CD6,ICD_DGNS_CD7,ICD_DGNS_VRSN_CD7,ICD_DGNS_CD8,ICD_DGNS_VRSN_CD8,ICD_DGNS_CD9,ICD_DGNS_VRSN_CD9,ICD_DGNS_CD10,ICD_DGNS_VRSN_CD10,ICD_DGNS_CD11,ICD_DGNS_VRSN_CD11,ICD_DGNS_CD12,ICD_DGNS_VRSN_CD12,CLM_CLNCL_TRIL_NUM,CARR_CLM_BLG_NPI_NUM,LINE_NUM,CARR_PRFRNG_PIN_NUM,PRF_PHYSN_UPIN,PRF_PHYSN_NPI,ORG_NPI_NUM,CARR_LINE_PRVDR_TYPE_CD,TAX_NUM,PRVDR_STATE_CD,PRVDR_ZIP,PRVDR_SPCLTY,PRTCPTNG_IND_CD,CARR_LINE_RDCD_PMT_PHYS_ASTN_C,LINE_SRVC_CNT,LINE_CMS_TYPE_SRVC_CD,LINE_PLACE_OF_SRVC_CD,CARR_LINE_PRCNG_LCLTY_CD,LINE_1ST_EXPNS_DT,LINE_LAST_EXPNS_DT,HCPCS_CD,HCPCS_1ST_MDFR_CD,HCPCS_2ND_MDFR_CD,BETOS_CD,LINE_NCH_PMT_AMT,LINE_BENE_PMT_AMT,LINE_PRVDR_PMT_AMT,LINE_BENE_PTB_DDCTBL_AMT,LINE_BENE_PRMRY_PYR_CD,LINE_BENE_PRMRY_PYR_PD_AMT,LINE_COINSRNC_AMT,LINE_SBMTD_CHRG_AMT,LINE_ALOWD_CHRG_AMT,LINE_PRCSG_IND_CD,LINE_PMT_80_100_CD,LINE_SERVICE_DEDUCTIBLE,CARR_LINE_MTUS_CNT,CARR_LINE_MTUS_CD,LINE_ICD_DGNS_CD,LINE_ICD_DGNS_VRSN_CD,HPSA_SCRCTY_IND_CD,CARR_LINE_RX_NUM,LINE_HCT_HGB_RSLT_NUM,LINE_HCT_HGB_TYPE_CD,LINE_NDC_CD,CARR_LINE_CLIA_LAB_NUM,CARR_LINE_ANSTHSA_UNIT_CNT,YR
0,-10000010254618,-10000930037915,O,71,2015-09-28,2015-09-28,02-Oct-2015,1,1,510,1,932.69,0.0,NaN,9999971093,A,932.69,0,1165.87,932.69,0.0,1,NaN,R4689,0,R4689,0,E781,0.0,J329,0.0,E119,0.0,D649,0.0,E089,0.0,B965,0.0,E848,0.0,E849,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1063654341,1,NaN,NaN,9999971093,1063654341,0,999349929,AL,361162501,1,1,0,10,1,11,1,28-Sep-2015,28-Sep-2015,NaN,NaN,NaN,NaN,91.04,0,91.04,0.0,,0,91.04,113.80,113.80,A,NaN,NaN,29,NaN,NaN,0,,NaN,0.0,R1,NaN,NaN,0,2015
1,-10000010254618,-10000930037915,O,71,2015-09-28,2015-09-28,02-Oct-2015,1,1,510,1,932.69,0.0,NaN,9999971093,A,932.69,0,1165.87,932.69,0.0,1,NaN,R4689,0,R4689,0,E781,0.0,J329,0.0,E119,0.0,D649,0.0,E089,0.0,B965,0.0,E848,0.0,E849,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1063654341,2,NaN,NaN,9999971093,1063654341,0,999349929,AL,361162501,1,1,0,10,1,11,1,28-Sep-2015,28-Sep-2015,NaN,NaN,NaN,NaN,108.80,0,108.80,0.0,,0,108.80,136.00,136.00,A,NaN,NaN,29,NaN,R4689,0,,NaN,0.0,R1,NaN,NaN,0,2015
2,-10000010254618,-10000930037915,O,71,2015-09-28,2015-09-28,02-Oct-2015,1,1,510,1,932.69,0.0,NaN,9999971093,A,932.69,0,1165.87,932.69,0.0,1,NaN,R4689,0,R4689,0,E781,0.0,J329,0.0,E119,0.0,D649,0.0,E089,0.0,B965,0.0,E848,0.0,E849,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1063654341,3,NaN,NaN,9999971093,1063654341,0,999349929,AL,361162501,1,1,0,10,1,11,1,28-Sep-2015,28-Sep-2015,NaN,NaN,NaN,NaN,108.80,0,108.80,0.0,,0,108.80,136.00,136.00,A,NaN,NaN,29,NaN,D649,0,,NaN,0.0,R1,NaN,NaN,0,2015
3,-10000010254618,-10000930037915,O,71,2015-09-28,2015-09-28,02-Oct-2015,1,1,510,1,932.69,0.0,NaN,9999971093,A,932.69,0,1165.87,932.69,0.0,1,NaN,R4689,0,R4689,0,E781,0.0,J329,0.0,E119,0.0,D649,0.0,E089,0.0,B965,0.0,E848,0.0,E849,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1063654341,4,NaN,NaN,9999971093,1063654341,0,999349929,AL,361162501,1,1,0,10,1,11,1,28-Sep-2015,28-Sep-2015,NaN,NaN,NaN,NaN,85.59,0,85.59,0.0,,0,85.59,106.99,106.99,A,NaN,NaN,29,NaN,R4689,0,,NaN,0.0,R1,NaN,NaN,0,2015
4,-10000010254618,-10000930037915,O,71,2015-09-28,2015-09-28,02-Oct-2015,1,1,510,1,932.69,0.0,NaN,9999971093,A,932.69,0,1165.87,932.69,0.0,1,NaN,R4689,0,R4689,0,E781,0.0,J329,0.0,E119,0.0,D649,0.0,E089,0.0,B965,0.0,E848,0.0,E849,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1063654341,5,NaN,NaN,9999971093,1063654341,0,999349929,AL,361162501,1,1,0,10,1,11,1,28-Sep-2015,28-Sep-2015,NaN,NaN,N

In [32]:
# keeping header information only (ie., line_num=1)
## why? mutliple rows per claim_id represent only one encounter with the same physician.
## ...See pg. 213 of ffs-codebook: https://www2.ccwdata.org/documents/10280/19022436/codebook-ffs-claims.pdf
professional = professional[professional['LINE_NUM'] == 1]
print(len(professional))

70159


In [33]:
diagnosis = professional[['BENE_ID', 'YR', 'PRNCPAL_DGNS_CD'] + [col for col in professional.columns if col.startswith('ICD_DGNS_CD')]]

# wide to long format data
diagnosis = pd.melt(diagnosis, id_vars=['BENE_ID', 'YR'], var_name='ICD_DIAG_COL', value_name='ICD_DIAG_CD')

# remove rows where ICD_DIAG_CD is not populated
diagnosis = diagnosis[diagnosis['ICD_DIAG_CD'].notna() & (diagnosis['ICD_DIAG_CD'] != '')]

# drop the 'ICD_DIAG_COL', then remove duplicates
diagnosis = diagnosis.drop(columns=['ICD_DIAG_COL'])
diagnosis = diagnosis.drop_duplicates()

diagnosis[diagnosis['BENE_ID'] == '-10000010254618'] #viewing a beneficiary

,BENE_ID,YR,ICD_DIAG_CD
0,-10000010254618,2015,R4689
1,-10000010254618,2016,R4689
2,-10000010254618,2017,R4689
3,-10000010254618,2018,R4689
4,-10000010254618,2019,R4689
...,...,...,...
771756,-10000010254618,2022,B965
841912,-10000010254618,2019,B965
841913,-10000010254618,2020,E848
841914,-10000010254618,2021,B965


In [34]:
# merge with ICD data file
diagnosis = pd.merge(diagnosis, LU_ICD, on='ICD_DIAG_CD', how='left')
num_diagnosis = diagnosis.groupby(['BENE_ID', 'YR']).size().reset_index(name='NUM_DIAG')
num_diagnosis.head(5)

,BENE_ID,YR,NUM_DIAG
0,-10000010254618,2015,9
1,-10000010254618,2016,9
2,-10000010254618,2017,11
3,-10000010254618,2018,11
4,-10000010254618,2019,13


In [35]:
professional_encounters = professional[['BENE_ID', 'CLM_ID', 'CLM_FROM_DT', 'CLM_THRU_DT', 'YR',
                                  'PRNCPAL_DGNS_CD', 'CLM_PMT_AMT']].drop_duplicates()

professional_encounters = professional_encounters.merge(num_diagnosis, on=['BENE_ID', 'YR'], how='left')
professional_encounters = professional_encounters.merge(LU_ICD, left_on='PRNCPAL_DGNS_CD', right_on='ICD_DIAG_CD',
                                                  how='left').drop(columns=['ICD_DIAG_CD'])
professional_encounters

,BENE_ID,CLM_ID,CLM_FROM_DT,CLM_THRU_DT,YR,PRNCPAL_DGNS_CD,CLM_PMT_AMT,NUM_DIAG,ICD_Description
0,-10000010254618,-10000930037915,2015-09-28,2015-09-28,2015,R4689,932.69,9,Other symptoms and signs involving appearance ...
1,-10000010254618,-10000930037916,2016-10-03,2016-10-03,2016,R4689,745.12,9,Other symptoms and signs involving appearance ...
2,-10000010254618,-10000930037917,2017-10-09,2017-10-09,2017,R4689,1040.54,11,Other symptoms and signs involving appearance ...
3,-10000010254618,-10000930037918,2018-10-15,2018-10-15,2018,R4689,1131.18,11,Other symptoms and signs involving appearance ...
4,-10000010254618,-10000930037919,2019-10-21,2019-10-21,2019,R4689,1382.27,13,Other symptoms and signs involving appearance ...
...,...,...,...,...,...,...,...,...,...
70154,-10000010288007,-10000931486358,2018-06-12,2018-06-12,2018,Z608,1953.80,12,Other problems related to social environment
70155,-10000010288007,-10000931486359,2019-06-18,2019-06-18,2019,Z608,1677.17,12,Other problems related to social environment
70156,-10000010288007,-10000931486360,2020-06-23,2020-06-23,2020,Z608,273.51,11,Other problems related to social environment
70157,-10000010288007,-10000931486361,2021-06-29,2021-06-29,2021,Z608,1323.33,11,Other problems related to social environment


In [36]:
professional_encounters.to_csv('/content/drive/MyDrive/Data/Output/professional_encounters.csv', index=False)
diagnosis.to_csv('/content/drive/MyDrive/Data/Output/professional_diagnosis.csv', index=False)